<a href="https://colab.research.google.com/github/dineshkumar-anandan/AWTFrame/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install paddlepaddle
!pip install paddleocr
!pip install paddlenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached protobuf-3.20.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.0 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.0 which is incompatible.
onnx 1.14.0 requires protobuf>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
googleapis-common-protos 1.59.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,

In [20]:
import paddle
import paddleocr
import torch
import logging

from paddlenlp import Taskflow
from concurrent.futures import ThreadPoolExecutor
from typing import List

In [21]:
def ernie(use_gpu: bool = True):
    return Taskflow("document_intelligence", lang="en",
                    FLAGS_eager_delete_scope=True,
                    FLAGS_eager_delete_tensor_gb=0.0,
                    FLAGS_memory_fraction_of_eager_deletion=1.0,
                    use_fast=True,
                    FLAGS_paddle_num_threads=5) if use_gpu else Taskflow(
        "document_intelligence", lang="en",
        device="cpu",
        FLAGS_eager_delete_scope=True,
        FLAGS_memory_fraction_of_eager_deletion=1.0,
        FLAGS_paddle_num_threads=5, use_fast=True)

In [22]:
def do_ernie_vqa(request_payloads: list, use_gpu: bool):
    taskflow_gpu = ernie(use_gpu=True)
    results = taskflow_gpu(request_payloads)
    responses_list = []
    try:
        results = [results] if len(request_payloads) == 1 else results
        for idx, result in enumerate(results):
            for res in result:
                response = extract_result(res)
                response['filename']=request_payloads[idx]["doc"]
                responses_list.append(response)
        return responses_list
    finally:
        del results
        del taskflow_gpu
        paddle.device.cuda.empty_cache()
        torch.cuda.empty_cache()
        logging.debug("Garbage collection  {}".format(gc.collect()))

In [23]:
def extract_result(res):
    response = {}
    values = res.get('result')
    predicted_values = []
    for value in values:
        node = {'predictedAttributionValue': value.get('value'), "score": value.get('prob')}
        predicted_values.append(node)
        if value.get("left") is not None and value.get("bottom") is not None and value.get(
                "right") is not None and value.get("top") is not None:
            node['leftPos'] = value.get('left')
            node['lowerPos'] = value.get('bottom')
            node['upperPos'] = value.get('top')
            node['rightPos'] = value.get('right')
            predicted_values.append(node)
    response.update({"question": res.get('prompt'), "predictions": predicted_values})
    return response

In [25]:
output = do_ernie_vqa([{"doc": "/content/pamela_wood.jpg", "prompt": "what is patient name"}], True)
logging.info("result completed")

[2023-05-07 11:59:00,128] [    INFO] - We are using <class 'paddlenlp.transformers.ernie_layout.tokenizer.ErnieLayoutTokenizer'> to load 'ernie-layoutx-base-uncased'.
[2023-05-07 11:59:00,129] [    INFO] - Already cached /root/.paddlenlp/models/ernie-layoutx-base-uncased/vocab.txt
[2023-05-07 11:59:00,135] [    INFO] - Already cached /root/.paddlenlp/models/ernie-layoutx-base-uncased/sentencepiece.bpe.model
[2023-05-07 11:59:00,893] [    INFO] - tokenizer config file saved in /root/.paddlenlp/models/ernie-layoutx-base-uncased/tokenizer_config.json
[2023-05-07 11:59:00,895] [    INFO] - Special tokens file saved in /root/.paddlenlp/models/ernie-layoutx-base-uncased/special_tokens_map.json


In [26]:
output

[{'question': 'what is patient name',
  'predictions': [{'predictedAttributionValue': 'PAMELA WOOD', 'score': 1.0}],
  'filename': '/content/pamela_wood.jpg'}]